# Eden AI

Eden AI is an AI consulting company that was founded to use its resources to empower people and create impactful products that use AI to improve the quality of life for individuals, businesses and societies at large.

This example goes over how to use LangChain to interact with Eden AI models



## Calling a model


Find a feature and model  on the EdenAI website.



### text generation

In [1]:
from langchain.llms import EdenAI
from langchain import PromptTemplate, LLMChain

llm=EdenAI(feature="text",provider="openai", params={"temperature" : 0.2,"max_tokens" : 250})

prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""

llm(prompt)

" No, a dog cannot drive a car.\n\nReasoning: \n1. Driving a car requires a valid driver's license, which dogs cannot obtain. \n2. Driving a car requires the ability to understand and follow traffic laws, which dogs cannot do. \n3. Driving a car requires the ability to operate a vehicle, which dogs cannot do. \n4. Therefore, a dog cannot drive a car."

### image generation

In [2]:
import base64
from io import BytesIO
from PIL import Image
import json
def print_base64_image(base64_string):
    # Decode the base64 string into binary data
    decoded_data = base64.b64decode(base64_string)

    # Create an in-memory stream to read the binary data
    image_stream = BytesIO(decoded_data)

    # Open the image using PIL
    image = Image.open(image_stream)

    # Display the image
    image.show()

In [5]:
text2image = EdenAI(
    feature="image" ,
    provider= "openai",
    params={
    "resolution" : "512x512"
    }
)

In [ ]:
image_output = text2image("A cat riding a motorcycle by Picasso")

In [8]:
print_base64_image(image_output)

### text generation with callback

In [9]:
from langchain.llms import EdenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = EdenAI(
    callbacks=[StreamingStdOutCallbackHandler()],
    feature="text",provider="openai", params={"temperature" : 0.2,"max_tokens" : 250}
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
print(llm(prompt))

 No, a dog cannot drive a car.

Reasoning:

1. A dog does not have the physical capability to operate a car.
2. A dog does not have the cognitive ability to understand the rules of the road and safely operate a car.
3. A dog does not have a valid driver's license.


## Chaining Calls

In [10]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [11]:
llm = EdenAI(
feature="text" ,provider="openai" , params={"temperature" : 0.2,"max_tokens" : 250}
)
text2image = EdenAI(
feature="image" ,provider="openai", params={"resolution" : "512x512"}
)

In [12]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
third_prompt = PromptTemplate(
    input_variables=["company_logo_description"],
    template="{company_logo_description}",
)
chain_three = LLMChain(llm=text2image, prompt=third_prompt)

In [20]:
# Run the chain specifying only the input variable for the first chain.
overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two, chain_three],verbose=True
)
output = overall_chain.run("hats")




> Entering new SimpleSequentialChain chain...


Headwear Haven
.

The logo for Headwear Haven should be a simple, yet eye-catching design. It should feature a stylized image of a hat, such as a baseball cap, with a bright, vibrant color palette. The design should be modern and minimalistic, with a few subtle details to make it stand out. The colors should be bold and inviting, and the overall look should be inviting and fun.
[1;...

> Finished chain.


In [21]:
#print the image
print_base64_image(output)